# Table pivoting using agate
### For when an ~~idiot~~ someone who thinks differently makes a spreadsheet, or otherwise if you need to flip data to be more convinient to work with for how you are using it

### Example case: Douglas County FY2017 salary data

In [1]:
#imports
import agate
DC = agate.Table.from_csv("data/douglas.csv")

When taking a first look at the data, you'll find that there isn't one row per employee. Instead, there is a row for each employee's salary components. Employees are listed multiple times, once for each salary component. In most spreadsheets, the salary components would be included as columns, not individual rows.

In [2]:
print(DC)

| column                      | data_type |
| --------------------------- | --------- |
| Employee Number             | Number    |
| Employee Last Name          | Text      |
| Employee First Name         | Text      |
| Employee Middle Name        | Text      |
| Department                  | Text      |
| Job                         | Text      |
| Termination Date            | Text      |
| Assignment Category Meaning | Text      |
| Employee Category Meaning   | Text      |
| Original Date of Hire       | Text      |
| Base Salary                 | Number    |
| 2017 Gross Pay              | Number    |
| Element Name                | Text      |
| Element Total               | Number    |
| INCLUDED_IN_BASE_PAY        | Boolean   |
| PAY_COMPONENTS              | Text      |
| Department_Rollup           | Text      |



More info if this example doesn't fit your needs: https://agate.readthedocs.io/en/1.6.1/api/table.html#agate.Table.pivot

### Step 1: Identify the key or keys

This should be the identifier number for what you are looking to group by, or in other words put only once in each data. Put everything in here that you want to group by. In this case, we want to group by employee. When looking at the data, we see some columns have are the same in every row for each employee. To include it in our final product, we should also group by these. The columns that are consistent for each row (with only the number of rows depending on how many pay components the employee has) that I also want included are: 'Employee Number','Employee Last Name', 'Employee First Name', 'Employee Middle Name', 'Department', 'Job', 'Assignment Category Meaning', 'Original Date of Hire', 'Base Salary', '2017 Gross Pay', 'Department_Rollup'. Pass these as a list in the first argument of .pivot()

### Step 2: Identify the column where you want to become headers

In our case, the pay components. These are in the column 'PAY_COMPONENTS'. That's our second argument. 'Element Namne' is too detailed for our purposes, but you could use that, too.

### Step 3: Let it know what we want to do with the column value turned header

We'll sum the 'Element Total' column. 

In [3]:
DC_pivot = DC.pivot(['Employee Number','Employee Last Name','Employee Middle Name','Employee First Name', 'Department', 'Job', 'Assignment Category Meaning', 'Original Date of Hire', 'Base Salary', '2017 Gross Pay', 'Department_Rollup'], 'PAY_COMPONENTS', aggregation=agate.Sum("Element Total"))

Print the columns. We now have a table with all of the keys we provided it with (From 'Employee Number' to '2017 Gross Pay') followed by all the values found in what was the 'PAY_COMPONENTS' column.

In [4]:
print(DC_pivot)

| column                      | data_type |
| --------------------------- | --------- |
| Employee Number             | Number    |
| Employee Last Name          | Text      |
| Employee Middle Name        | Text      |
| Employee First Name         | Text      |
| Department                  | Text      |
| Job                         | Text      |
| Assignment Category Meaning | Text      |
| Original Date of Hire       | Text      |
| Base Salary                 | Number    |
| 2017 Gross Pay              | Number    |
| Department_Rollup           | Text      |
| Longevity                   | Number    |
| Base Pay                    | Number    |
| Overtime                    | Number    |
| Other                       | Number    |
| Education stipend           | Number    |
| Specialty                   | Number    |
| Comp time cashout           | Number    |
| Annual Payoff               | Number    |



### Step 4: Check the data entegrity.

If done correctly, the row count of the new table will match the number of employees in the original table. In other words, employees won't be listed twice anymore. One way of doing this is by grouping again.

In [5]:
#Group and count
by_id = DC_pivot.group_by('Employee Number')
DC_pivot_group = by_id.aggregate([
    ('count', agate.Count())
])

Is the lenth of this equal to what we grouped?

In [6]:
print(len(DC_pivot_group))
print(len(DC_pivot))

2805
2806


Well, darn. Let's look at who is listed twice and why.

In [7]:
DC_pivot_group.order_by('count', reverse=True).print_table()

| Employee Number | count |
| --------------- | ----- |
|           2,076 |     2 |
|             249 |     1 |
|             250 |     1 |
|             251 |     1 |
|             252 |     1 |
|             254 |     1 |
|             255 |     1 |
|             261 |     1 |
|             265 |     1 |
|             266 |     1 |
|             269 |     1 |
|             272 |     1 |
|             273 |     1 |
|             282 |     1 |
|             284 |     1 |
|             292 |     1 |
|             295 |     1 |
|             296 |     1 |
|             301 |     1 |
|             308 |     1 |
|             ... |   ... |


Employee 2076 is listed twice in our data. When looking, we see that this is because she has two different positions with different salaries. We'll handle this somehow, but it's a seperate problem, not a problem with the grouping

I still want to know if INCLUDED_IN_BASE_PAY is consistent depending on what the Element Name is. I did some coding which I left out, and it is.

### Export your new data and get to the actual work!

In [8]:
DC_pivot.to_csv("Q:/EDITORIAL-Q/1Salaries/2017/DC_redone.csv")